In [5]:
import re
import requests
import time
from dotenv import load_dotenv
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup

## 

---

In [15]:
# Initailize
jobDescription = ["software",
                  "software developer",
                  "software dev"
                  "SDET",
                  "軟體實習生",
                  "軟體測試實習生",
                  "data analytics",
                  "data science",
                  "data scientist",
                  "AI"]

# 搜索的內容
searchKeyWords = ["software intern", "軟體實習生", "後端實習生", "Node.js intern"]


In [43]:
def containOrNot(job, jobDescription: list):
    """
    params:
    - job: {
        type: class,
        data: class get by driver
        }
    - jobDescription: {
        type: list,
        data: list of job description
    }
    - jobDict: {
        type: list of dictionary
    }
    
    """
    tokenList = [string.lower() for string in job.get_attribute("data-job-name").split(" ")]
    for token in tokenList:
        if token in jobDescription and "intern" in tokenList or "實習" in tokenList:
            return True
    return False
                   
def findPageNumber(driver):
    pageNumber = driver.find_element(By.CLASS_NAME, "js-paging-select").find_element(By.TAG_NAME, "option").text.split(" / ")[1][:3]# 150頁    
    return int(pageNumber)

### Problem record
- 爬完自動切換頁面時後會失敗


In [51]:
driver = webdriver.Chrome()

# 進入104 首頁
# driver.get("https://www.google.com/")
driver.get("https://www.104.com.tw/jobs/main/")
driver.implicitly_wait(2) # Loading wait⋯

# 輸入要查詢的工作內容
keyWords = driver.find_element(By.CLASS_NAME, "form-control")
keyWords.send_keys("test")
searchBtn = driver.find_element(By.CLASS_NAME, "btn-secondary")
searchBtn.click()

# 開始搜索符合條件的工作
jobs = [] # array to store dict results
for keyword in searchKeyWords:
    print("------------{}------------".format(keyword))
    searchBox = driver.find_element(By.ID, "keyword")
    searchBox.clear()
    searchBox.send_keys(keyword)
    searchBoxEnter = driver.find_element(By.CLASS_NAME, "b-btn--primary")
    searchBoxEnter.click()
    
    currentPage = 0
    pageNumber = findPageNumber(driver)

    while currentPage != 5:
        jobs_parent = driver.find_element(By.ID, "js-job-content")
        jobs_per_page = jobs_parent.find_elements(By.TAG_NAME, "article")
        for job in jobs_per_page:
            if containOrNot(job, jobDescription):
                jobNameList = [i["name"] for i in jobs]
                if job.get_attribute("data-job-name") not in jobNameList:
                    jobs.append({
                        "name": job.get_attribute("data-job-name"),
                        "company": job.get_attribute("data-cust-name")
                    }) 
                    print(job.get_attribute("data-job-name"))
                    
        time.sleep(2)
        nextPageBtn = driver.find_element(By.CLASS_NAME, "js-next-page")
        nextPageBtn.click()
        currentPage += 1
    
# Save the results
dfJobs = pd.DataFrame(jobs)
dfJobs.to_csv("test.csv")

# end
driver.close()

------------software intern------------
AI Software Intern 人工智慧軟體實習生 (台北)
Software Engineer, Intern / Part-time
Software Engineer Intern /軟體開發實習生
Software R&D intern (應屆畢業生有機會轉正聘用)
實習軟體工程師 Software Engineer Intern
〔Student Intern 學生實習專區〕System Software Engineer Intern- GPU Server
Software Engineer Dev Intern (12 months at least, can start within 1 month)
[學生實習 Student Intern ] System Software Engineering Intern - GPU- Taipei
軟體開發工讀生 Software Developer Intern
------------軟體實習生------------
------------後端實習生------------


ElementClickInterceptedException: Message: element click intercepted: Element <button class="b-btn b-btn--primary is-large gtm-main-search" type="submit">...</button> is not clickable at point (1027, 72). Other element would receive the click: <div class="b-block--loading" id="js-action-overlay" style="">...</div>
  (Session info: chrome=122.0.6261.57)
Stacktrace:
0   chromedriver                        0x00000001010e8580 chromedriver + 3966336
1   chromedriver                        0x00000001010e0b0c chromedriver + 3934988
2   chromedriver                        0x0000000100d63da0 chromedriver + 277920
3   chromedriver                        0x0000000100dabe68 chromedriver + 573032
4   chromedriver                        0x0000000100daa3ec chromedriver + 566252
5   chromedriver                        0x0000000100da856c chromedriver + 558444
6   chromedriver                        0x0000000100da7a9c chromedriver + 555676
7   chromedriver                        0x0000000100d9ce84 chrom

In [ ]:
dfJobs = pd.DataFrame(jobs)
dfJobs
# dfJobs.to_csv("./test.csv")

,name,company
0,"Software Engineer, Intern / Part-time",臺灣發展軟體科技股份有限公司
1,Software R&amp;D intern (應屆畢業生有機會轉正聘用),美商海鸚科技股份有限公司台灣分公司
2,Software Engineer Intern / 軟體開發實習生,自由食間_滿食空股份有限公司
3,Software Engineer Dev Intern (12 months at lea...,英屬開曼群島商庫幣科技有限公司台灣分公司
4,"Software Engineer, Intern / Part-time",臺灣發展軟體科技股份有限公司
5,Software R&amp;D intern (應屆畢業生有機會轉正聘用),美商海鸚科技股份有限公司台灣分公司
6,Software Engineer Intern / 軟體開發實習生,自由食間_滿食空股份有限公司
7,Software Engineer Dev Intern (12 months at lea...,英屬開曼群島商庫幣科技有限公司台灣分公司
8,Software R&D Intern 軟體研發實習生【Quanta Vision 2024...,光大科技股份有限公司
9,Software Engineering Intern (QA),CommScope Ruckus_台灣康普通信系統有限公司


---

In [ ]:
driver = webdriver.Chrome()

# 進入104 首頁
# driver.get("https://www.google.com/")
driver.get("https://www.104.com.tw/jobs/main/")
driver.implicitly_wait(2) # Loading wait⋯

# 輸入要查詢的工作內容
keyWords = driver.find_element(By.CLASS_NAME, "form-control")
keyWords.send_keys("software intern")
searchBtn = driver.find_element(By.CLASS_NAME, "btn-secondary")
searchBtn.click()

currentPage = driver.find_element(By.CLASS_NAME, "js-paging-select").find_element(By.TAG_NAME, "option").text
print(currentPage)

第 1 / 150 頁


In [ ]:
import re

# 假設你已經有了字符串 '150 頁'
text = '150 頁'

# 使用正規表達式找到字符串中的數字部分
match = re.search(r'\d+', text)

# 如果找到了數字，就取出來
if match:
    number = match.group()
    print(number)

150


In [37]:
jobs

[{'name': 'AI Software Intern 人工智慧軟體實習生 (台北)', 'company': '立達軟體科技股份有限公司'}]

In [48]:
aDict = [{"a": 1}, {"b": 2}]
aList = [ for i in aDict]
aList

[{'a': 1}, {'b': 2}]

In [50]:
jobs
[]
jobNameList = [i["name"] for i in jobs]
jobNameList

['AI Software Intern 人工智慧軟體實習生 (台北)']